## Explore and cluster the neighborhoods in Toronto

In [45]:
# The code was removed by Watson Studio for sharing.

In [3]:
import numpy as np 

!conda install -c conda-forge pandas
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

from geopy.geocoders import Nominatim 

import requests 
#from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium 

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
pandas                    0.23.4           py35hf8a1672_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


In [46]:
df=pd.read_csv(project.get_file('toronto.csv'))
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]))

The dataframe has 12 boroughs and 103 neighborhoods.


In [47]:
df.head()

,PostalCode,Borough,Neighborhood,lat,lng
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662843,-79.389638


Find the latitude and longitude values of Toronto

In [48]:
address = 'Toronto, ON'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Keep the data only "boroughs" that contain the word Toronto 

In [49]:
toronto_data = df[df['Borough'].str.contains('Toronto', regex=False)].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,lat,lng
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [51]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, label in zip(toronto_data['lat'], toronto_data['lng'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='Brown',
        fill=True,
        fill_opacity=0.8,
        ).add_to(map_toronto)  
    
map_toronto

Explore API in Foursquare

In [52]:
# The code was removed by Watson Studio for sharing.

Get nearby Venues

In [53]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [54]:
LIMIT=100
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['lat'],
                                   longitudes=toronto_data['lng']
                                  )

Regent Park , Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond , Adelaide , King
Dufferin , Dovercourt Village
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
The Danforth West , Riverdale
Toronto Dominion Centre , Design Exchange
Brockton , Parkdale Village , Exhibition Place
India Bazaar , The Beaches West
Commerce Court , Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park , The Junction South
North Toronto West
The Annex , North Midtown , Yorkville
Parkdale , Roncesvalles
Davisville
University of Toronto , Harbord
Runnymede , Swansea
Moore Park , Summerhill East
Kensington Market , Chinatown , Grange Park
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Rosedale
Stn A PO Boxes25 The Esplanade Enclave of 

Delete the Venues named 'Neighborhood'

In [55]:
toronto_venues = toronto_venues[toronto_venues['Venue Category'] != 'Neighborhood']

In [56]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.insert (0,'Neighborhood',toronto_venues['Neighborhood'])
toronto_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pilates Studio,Pizza Place,Playground,Plaza,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rest Area,Restaurant,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store,Yoga Studio
0,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [57]:
toronto_onehot.shape

(2388, 244)

In [58]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pilates Studio,Pizza Place,Playground,Plaza,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rest Area,Restaurant,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.017544,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.0,0.017544,0.035088,0.0,0.000000,0.0,0.0,0.017544,0.017544,0.035088,0.0,0.017544,0.0,0.017544,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.035088,0.0,0.0,0.000000,0.035088,0.000000,0.0,0.0,0.017544,0.052632,0.070175,0.0,0.0,0.0,0.00,0.017544,0.000000,0.017544,0.000000,0.017544,0.0,0.017544,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.017544,0.000000,0.000000,0.00

In [59]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1          Restaurant  0.05
2        Cocktail Bar  0.05
3  Seafood Restaurant  0.04
4      Farmers Market  0.04


----Brockton , Parkdale Village , Exhibition Place----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.06
2         Park  0.06
3       Bakery  0.04
4   Restaurant  0.03


----Business reply mail Processing Centre969 Eastern Enclave of M4L----
                venue  freq
0                 Spa  0.06
1  Light Rail Station  0.06
2          Comic Shop  0.06
3    Recording Studio  0.06
4          Restaurant  0.06


----CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport----
                venue  freq
0          Restaurant  0.07
1         Coffee Shop  0.06
2      Sandwich Place  0.04
3  Italian Restaurant  0.04
4               Hotel  0.04


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.16
1   Itali

In [60]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [61]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Sushi Restaurant,Café,Farmers Market,Seafood Restaurant,Cheese Shop,Beer Bar,Bakery
1,"Brockton , Parkdale Village , Exhibition Place",Coffee Shop,Café,Park,Bakery,Grocery Store,Bar,Burger Joint,Men's Store,Gym / Fitness Center,Pet Store
2,Business reply mail Processing Centre969 Easte...,Comic Shop,Recording Studio,Farmers Market,Fast Food Restaurant,Skate Park,Spa,Burrito Place,Light Rail Station,Butcher,Moving Target
3,"CN Tower , King and Spadina , Railway Lands , ...",Restaurant,Coffee Shop,Italian Restaurant,Hotel,Sandwich Place,French Restaurant,Gym,Bar,Vegetarian / Vegan Restaurant,Pub
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Bubble Tea Shop,Burger Joint,Sandwich Place,Japanese Restaurant,Bar,Salad Place,Spa


K-means cluster

In [62]:
kclusters=5
toronto_grouped_clustering = toronto_grouped.drop(columns='Neighborhood')
kmeans_tot = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans_tot.cluster_centers_

array([[ 0.00030303,  0.0003651 ,  0.0003651 , ...,  0.00247564,
         0.00256045,  0.00555735],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [63]:
toronto_merged = toronto_data
toronto_merged['Cluster Labels'] = kmeans_tot.labels_
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head()

,PostalCode,Borough,Neighborhood,lat,lng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Café,Breakfast Spot,Pub,Mexican Restaurant,Theater,Chocolate Shop,Restaurant
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Thai Restaurant,Bakery,Bubble Tea Shop,Diner
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Restaurant,Café,Clothing Store,Hotel,Cocktail Bar,Gastropub,Cosmetics Shop,Japanese Restaurant,American Restaurant
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Playground,Pub,Coffee Shop,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Event Space
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Restaurant,Cocktail Bar,Sushi Restaurant,Café,Farmers Market,Seafood Restaurant,Cheese Shop,Beer Bar,Bakery


In [64]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['lat'], toronto_merged['lng'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [65]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park , Harbourfront",Coffee Shop,Park,Bakery,Café,Breakfast Spot,Pub,Mexican Restaurant,Theater,Chocolate Shop,Restaurant
1,"Garden District, Ryerson",Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Thai Restaurant,Bakery,Bubble Tea Shop,Diner
2,St. James Town,Coffee Shop,Restaurant,Café,Clothing Store,Hotel,Cocktail Bar,Gastropub,Cosmetics Shop,Japanese Restaurant,American Restaurant
3,The Beaches,Playground,Pub,Coffee Shop,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Event Space
4,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Sushi Restaurant,Café,Farmers Market,Seafood Restaurant,Cheese Shop,Beer Bar,Bakery
5,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Bubble Tea Shop,Burger Joint,Sandwich Place,Japanese Restaurant,Bar,Salad Place,Spa
6,Christie,Café,Grocery Store,Park,Diner,Coffee Shop,Baby Store,Italian Restaurant,Restaurant,Athletics & Sports,Nightclub
7,"Richmond , Adelaide , King",Coffee Shop,Café,Steakhouse,Thai Restaurant,American Restaurant,Bar,Breakfast Spot,Restaurant,Burger Joint,Clothing Store
8,"Dufferin , Dovercourt Village",Bakery,Café,Bar,Beer Store,Coffee Shop,Pizza Place,Fast Food Restaurant,Beer Bar,Food Truck,Flower Shop
9,"Harbourfront East , Union Station , Toronto Is...",Coffee Shop,Hotel,Pizza Place,Café,Aquarium,Sports Bar,Scenic Lookout,Italian Restaurant,Brewery,Fried Chicken Joint


In [66]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,"Toronto Dominion Centre , Design Exchange",Coffee Shop,Hotel,Café,Gastropub,Sports Bar,American Restaurant,Restaurant,Italian Restaurant,Bakery,Deli / Bodega
25,Davisville,Restaurant,Bakery,Italian Restaurant,Yoga Studio,Coffee Shop,Bar,Spa,Pizza Place,Furniture / Home Store,Sandwich Place


In [67]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,"University of Toronto , Harbord",Café,Coffee Shop,Bar,Bookstore,Restaurant,Bakery,Japanese Restaurant,Italian Restaurant,Nightclub,Noodle House


In [68]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,"St. James Town , Cabbagetown",Coffee Shop,Restaurant,Chinese Restaurant,Indian Restaurant,Italian Restaurant,Pizza Place,Pub,Bakery,Market,Café


In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Roselawn,Ice Cream Shop,Home Service,Garden,Yoga Studio,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space


## K-means for both cities together

In [132]:
nyc_grouped = pd.read_csv(project.get_file('nyc.csv'))
nyc_grouped.drop(nyc_grouped.columns[[0]],axis=1,inplace = True)

In [133]:
nyc_grouped['country'] = 'US'
toronto_grouped['country'] = 'CAN'

In [135]:
toronto_nyc=nyc_grouped.append(toronto_grouped,ignore_index=True,sort=True)

#merge neighborhood and country
toronto_nyc['Neighborhood'] = toronto_nyc['Neighborhood'].map(str) + ['-']+toronto_nyc['country']
toronto_nyc.drop(columns='country',inplace = True)

df1=toronto_nyc['Neighborhood']
toronto_nyc.drop(columns='Neighborhood',inplace = True)
toronto_nyc.insert (0,'Neighborhood',df1)
toronto_nyc.fillna(0,inplace=True)

toronto_nyc.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Green,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pop-Up Shop,Portuguese Restaurant,Poutine Place,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rive

In [136]:
print(toronto_nyc.shape,toronto_grouped.shape,nyc_group.shape)

(78, 359) (38, 245) (40, 330)


In [137]:
all_clustering_all = toronto_nyc.drop(columns=['Neighborhood'])

kmeans_all = KMeans(n_clusters=5, random_state=0).fit(all_clustering)

kmeans_all.cluster_centers_

array([[  3.90449438e-03,  -5.42101086e-20,  -5.42101086e-20, ...,
          1.08420217e-19,   2.65449438e-03,   6.56250000e-03],
       [  2.32558140e-04,   2.80190530e-04,   2.80190530e-04, ...,
         -1.62630326e-19,   2.66986576e-03,   5.19517584e-03],
       [  1.11111111e-03,  -5.42101086e-20,   1.11111111e-03, ...,
          1.70940171e-03,   5.15482696e-03,   1.78304846e-02],
       [  0.00000000e+00,   1.11111111e-03,  -5.42101086e-20, ...,
          1.11111111e-03,   8.88888889e-03,   8.88888889e-03],
       [  1.11111111e-03,  -5.42101086e-20,  -5.42101086e-20, ...,
          1.11111111e-03,   5.55555556e-03,   1.49425287e-02]])

In [138]:
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

all_venues_sorted = pd.DataFrame(columns=columns)
all_venues_sorted['Neighborhood'] = toronto_nyc['Neighborhood']

for ind in np.arange(toronto_nyc.shape[0]):
    all_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_nyc.iloc[ind, :], num_top_venues)

all_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City-US,Park,Coffee Shop,Hotel,Historic Site,Burger Joint,Memorial Site,Italian Restaurant,Wine Shop,American Restaurant,Pizza Place
1,Carnegie Hill-US,Pizza Place,Cosmetics Shop,Coffee Shop,Café,Yoga Studio,Bar,Wine Shop,Bookstore,Spa,Gym / Fitness Center
2,Central Harlem-US,African Restaurant,Chinese Restaurant,American Restaurant,Seafood Restaurant,Cosmetics Shop,French Restaurant,Public Art,Pizza Place,Caribbean Restaurant,Park
3,Chelsea-US,Coffee Shop,Italian Restaurant,Bakery,Nightclub,Ice Cream Shop,Seafood Restaurant,Art Gallery,Theater,Hotel,American Restaurant
4,Chinatown-US,Chinese Restaurant,Dim Sum Restaurant,American Restaurant,Vietnamese Restaurant,Ice Cream Shop,Dumpling Restaurant,Cocktail Bar,Bakery,Hotpot Restaurant,Bubble Tea Shop


In [139]:
all_merged = toronto_nyc[['Neighborhood']]
all_merged['Cluster Labels'] = kmeans_all.labels_
all_merged = all_merged.join(all_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
all_merged.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City-US,1,Park,Coffee Shop,Hotel,Historic Site,Burger Joint,Memorial Site,Italian Restaurant,Wine Shop,American Restaurant,Pizza Place
1,Carnegie Hill-US,1,Pizza Place,Cosmetics Shop,Coffee Shop,Café,Yoga Studio,Bar,Wine Shop,Bookstore,Spa,Gym / Fitness Center
2,Central Harlem-US,1,African Restaurant,Chinese Restaurant,American Restaurant,Seafood Restaurant,Cosmetics Shop,French Restaurant,Public Art,Pizza Place,Caribbean Restaurant,Park
3,Chelsea-US,1,Coffee Shop,Italian Restaurant,Bakery,Nightclub,Ice Cream Shop,Seafood Restaurant,Art Gallery,Theater,Hotel,American Restaurant
4,Chinatown-US,1,Chinese Restaurant,Dim Sum Restaurant,American Restaurant,Vietnamese Restaurant,Ice Cream Shop,Dumpling Restaurant,Cocktail Bar,Bakery,Hotpot Restaurant,Bubble Tea Shop


In [142]:
all_merged.loc[all_merged['Cluster Labels'] == 0, all_merged.columns[[0] + list(range(2, all_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Tribeca-US,Italian Restaurant,American Restaurant,Spa,Park,Cocktail Bar,Wine Bar,Gym,Greek Restaurant,Boutique,Café
33,Tudor City-US,Park,Café,Pizza Place,Mexican Restaurant,Diner,Dog Run,Greek Restaurant,Spa,Restaurant,Sushi Restaurant
34,Turtle Bay-US,Italian Restaurant,Sushi Restaurant,Wine Bar,Coffee Shop,Steakhouse,Japanese Restaurant,Hotel,French Restaurant,Noodle House,Café
35,Upper East Side-US,Italian Restaurant,Exhibit,Juice Bar,Art Gallery,American Restaurant,Bakery,French Restaurant,Boutique,Spa,Gym / Fitness Center
36,Upper West Side-US,Italian Restaurant,Wine Bar,Bar,Indian Restaurant,Bakery,Coffee Shop,Burger Joint,Vegetarian / Vegan Restaurant,Ice Cream Shop,Mediterranean Restaurant
37,Washington Heights-US,Café,Bakery,Sandwich Place,Grocery Store,Deli / Bodega,Supermarket,Tapas Restaurant,Latin American Restaurant,Mexican Restaurant,Mobile Phone Shop
38,West Village-US,Italian Restaurant,New American Restaurant,Cosmetics Shop,Gastropub,Jazz Club,Wine Bar,American Restaurant,French Restaurant,Ice Cream Shop,Bakery
39,Yorkville-US,Coffee Shop,Bar,Italian Restaurant,Gym,Sushi Restaurant,Pizza Place,Japanese Restaurant,Thai Restaurant,Deli / Bodega,Diner


In [143]:
all_merged.loc[all_merged['Cluster Labels'] == 1, all_merged.columns[[0] + list(range(2, all_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City-US,Park,Coffee Shop,Hotel,Historic Site,Burger Joint,Memorial Site,Italian Restaurant,Wine Shop,American Restaurant,Pizza Place
1,Carnegie Hill-US,Pizza Place,Cosmetics Shop,Coffee Shop,Café,Yoga Studio,Bar,Wine Shop,Bookstore,Spa,Gym / Fitness Center
2,Central Harlem-US,African Restaurant,Chinese Restaurant,American Restaurant,Seafood Restaurant,Cosmetics Shop,French Restaurant,Public Art,Pizza Place,Caribbean Restaurant,Park
3,Chelsea-US,Coffee Shop,Italian Restaurant,Bakery,Nightclub,Ice Cream Shop,Seafood Restaurant,Art Gallery,Theater,Hotel,American Restaurant
4,Chinatown-US,Chinese Restaurant,Dim Sum Restaurant,American Restaurant,Vietnamese Restaurant,Ice Cream Shop,Dumpling Restaurant,Cocktail Bar,Bakery,Hotpot Restaurant,Bubble Tea Shop
40,Berczy Park-CAN,Coffee Shop,Cocktail Bar,Restaurant,Cheese Shop,Bakery,Beer Bar,Sushi Restaurant,Steakhouse,Farmers Market,Seafood Restaurant
41,"Brockton , Parkdale Village , Exhibition Place...",Coffee Shop,Café,Park,Bakery,Pet Store,Grocery Store,Restaurant,Bar,Burger Joint,Men's Store
42,Business reply mail Processing Centre969 Easte...,Moving Target,Skate Park,Butcher,Garden,Park,Garden Center,Restaurant,Burrito Place,Brewery,Auto Workshop
43,"CN Tower , King and Spadina , Railway Lands , ...",Restaurant,Coffee Shop,Hotel,Italian Restaurant,Sandwich Place,Gym,Bar,French Restaurant,Vegetarian / Vegan Restaurant,Yoga Studio
44,Central Bay Street-CAN,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Bar,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Indian Restaurant,Falafel Restaurant


In [144]:
all_merged.loc[all_merged['Cluster Labels'] == 3, all_merged.columns[[0] + list(range(2, all_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Midtown-US,Hotel,Theater,Bakery,Sporting Goods Shop,Steakhouse,Clothing Store,Gym / Fitness Center,Food Truck,Park,Coffee Shop
24,Midtown South-US,Korean Restaurant,Coffee Shop,Cosmetics Shop,Hotel Bar,Hotel,Italian Restaurant,Boutique,Gym / Fitness Center,Japanese Restaurant,Scenic Lookout
25,Morningside Heights-US,Coffee Shop,Park,American Restaurant,Bookstore,Grocery Store,Deli / Bodega,Burger Joint,Tennis Court,Sandwich Place,Seafood Restaurant
26,Murray Hill-US,Hotel,Sandwich Place,Salon / Barbershop,Burger Joint,Bar,Italian Restaurant,Japanese Restaurant,American Restaurant,French Restaurant,Gym / Fitness Center
27,Noho-US,Italian Restaurant,French Restaurant,Cocktail Bar,Coffee Shop,Sushi Restaurant,Grocery Store,Gift Shop,Mexican Restaurant,Bookstore,Hotel
28,Roosevelt Island-US,Park,Sandwich Place,Dog Run,Liquor Store,Greek Restaurant,Kosher Restaurant,Bridge,Outdoors & Recreation,School,Gym
29,Soho-US,Clothing Store,Boutique,Women's Store,Men's Store,Shoe Store,Coffee Shop,Mediterranean Restaurant,French Restaurant,Art Gallery,Italian Restaurant
30,Stuyvesant Town-US,Bar,Park,Heliport,Baseball Field,Gas Station,Pet Service,Cocktail Bar,Coffee Shop,Basketball Court,Harbor / Marina
31,Sutton Place-US,Gym / Fitness Center,Italian Restaurant,Indian Restaurant,Furniture / Home Store,Juice Bar,Ice Cream Shop,Grocery Store,Dessert Shop,Bakery,American Restaurant


In [145]:
all_merged.loc[all_merged['Cluster Labels'] == 4, all_merged.columns[[0] + list(range(2, all_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Civic Center-US,Gym / Fitness Center,Bakery,Cocktail Bar,Italian Restaurant,Yoga Studio,Spa,Park,Sporting Goods Shop,Sushi Restaurant,Sandwich Place
6,Clinton-US,Theater,American Restaurant,Italian Restaurant,Hotel,Gym / Fitness Center,Wine Shop,Spa,Coffee Shop,Dog Run,Sandwich Place
7,East Harlem-US,Mexican Restaurant,Bakery,Latin American Restaurant,Thai Restaurant,Deli / Bodega,Sandwich Place,Pizza Place,Liquor Store,Bank,Beer Bar
8,East Village-US,Ice Cream Shop,Bar,Wine Bar,Mexican Restaurant,Cocktail Bar,Vegetarian / Vegan Restaurant,Café,Pizza Place,Chinese Restaurant,Speakeasy
9,Financial District-US,Coffee Shop,Hotel,Wine Shop,Steakhouse,Juice Bar,Park,Gym / Fitness Center,Gym,Bar,Pizza Place
10,Flatiron-US,Gym / Fitness Center,Yoga Studio,American Restaurant,Spa,Women's Store,Cosmetics Shop,Salon / Barbershop,New American Restaurant,Mexican Restaurant,Japanese Restaurant
11,Gramercy-US,Italian Restaurant,Bagel Shop,Thrift / Vintage Store,Cocktail Bar,Wine Shop,Restaurant,American Restaurant,Vietnamese Restaurant,Comedy Club,Pizza Place
12,Greenwich Village-US,Italian Restaurant,Clothing Store,French Restaurant,Sushi Restaurant,Café,Indian Restaurant,Boutique,Bakery,American Restaurant,Seafood Restaurant
13,Hamilton Heights-US,Mexican Restaurant,Café,Pizza Place,Yoga Studio,Sushi Restaurant,Bakery,Caribbean Restaurant,Chinese Restaurant,Cocktail Bar,Coffee Shop
